In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [5]:
sunscreens_url = []
for i in range (1, 17):
    url = f'https://www.yesstyle.com/en/women-sun-care/list.html/bcc.15396_bpt.46#/bpt=46&sb=136&bt=37&bcc=15396&s=10&pn={i}&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    sunscreens_list = soup.find_all('div', class_='itemContainer')
    for item in sunscreens_list:
        for link in item.find_all('a', href=True): 
            sunscreens_url.append(link['href'])


In [6]:
print(len(sunscreens_url))

576


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [51]:
"""
test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text
"""

In [6]:
sunscreens_url[467]

'https://www.yesstyle.com/en/pyunkang-yul-eye-cream-50ml-1ml-x-50pcs/info.html/pid.1067160762'

In [10]:
sunscreens = []

In [15]:
for link in sunscreens_url[303:576]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text.strip() #ingredients
        rating = soup.find('div', class_='satisfied').text.strip()
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    sunscreens_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    sunscreens.append(sunscreens_info)

In [16]:
print(len(sunscreens))

576


In [81]:
sunscreens[100]

{'name': '\nKUMANO COSME - Pharmaact Deep Cleansing Oil\n',
 'rating': '97.3% Clientes satisfechos|317 reseñas',
 'ingredients': '\nMineral oil, PEG-8 glyceryl isostearate, water, glycerin, dimethicone, citric acid, jojoba oil, squalane, ethyl paraben, perfume\n\nMás\n\n'}

## 3. PASARLO A DATAFRAME

In [17]:
sunscreens_df = pd.DataFrame(sunscreens)
sunscreens_df

,name,rating,ingredients
0,\nMISSHA - All-Around Safe Block Essence Sun S...,"97.8% customers satisfied|1,243 reviews","Water, Ethylhexyl Methoxycinnamate, Ethylhexyl..."
1,\nCOSRX - Aloe Soothing Sun Cream\n,"97.1% customers satisfied|3,975 reviews","Water, Ethylhexyl Methoxycinnamate, Glycerin, ..."
2,\nCanmake - Mermaid Skin Gel UV SPF 50+ PA++++...,98.3% customers satisfied|382 reviews,"Water, Ethylhexyl Methoxycinnamate, BG, Diethy..."
3,\nJigott - UV Sun Block SPF 50+ PA+++ 70ml - 2...,"95.4% customers satisfied|1,308 reviews","Water, Ethylhexyl Methoxycinnamate, Glyceryl S..."
4,\nMISSHA - All-Around Safe Block Essence Sun M...,97.4% customers satisfied|613 reviews,"Helichrysum Italicum Extract, Ethylhexyl Metho..."
...,...,...,...
571,\nShiseido - Anessa Perfect UV Sunscreen Skinc...,95.9% customers satisfied|49 reviews,"Dimethicone, water, zinc oxide, ethanol, ethyl..."
572,\nLABIOTTE - UV Veil Moisture Sun Essence Spec...,100% customers satisfied|124 reviews,"Water, Homosalate, Butylene Glycol, Octocrylen..."
573,\ncoxir - Ceramide Daily UV Sun Block\n,88.8% customers satisfied|12 reviews,"Water, Micro Particle Titanium Dioxide, Alumin..."
574,\nShiseido - Anessa Perfect UV Sunscreen Skinc...,83.3% customers satisfied|7 reviews,"Water, dimethicone, zinc oxide, ethylhexyl met..."


In [19]:
sunscreens = sunscreens_df.to_csv('yesstyle_data/sunscreens.csv')